In [1]:
from datasets import load_dataset
import pandas as pd
import spacy as sp
from spacy.lang.en.stop_words import STOP_WORDS
import numpy as np


import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis.gensim
import pyLDAvis
from nltk.stem import WordNetLemmatizer

/Users/shafayetulislam/Documents/mentalHealthTM/mentalHealthTMenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [3]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from IPython.display import display

In [25]:
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [6]:
# view the col context fully...

pd.set_option('display.max_colwidth', None)

In [7]:
ds = load_dataset("Amod/mental_health_counseling_conversations")

In [8]:
df = ds['train'].to_pandas()

In [9]:
unique_contexts = df['Context'].drop_duplicates()

In [10]:
unique_contexts_list = unique_contexts.tolist()

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shafayetulislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def simple_preprocess(text):
    """
    Simple preprocessing function without relying on NLTK
    """
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Split on whitespace
    tokens = text.split()
    
    
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    
    return ' '.join(tokens)

In [26]:
def perform_topic_modeling(texts, n_topics=5):
    """
    Perform topic modeling using both LDA and NMF
    """
    # Preprocess texts
    processed_texts = [simple_preprocess(text) for text in texts]
    
    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer(
        max_features=1000,
        min_df=2,
        max_df=0.95
    )
    dtm = vectorizer.fit_transform(processed_texts)
    
    # Perform LDA
    lda_model = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        max_iter=20
    )
    
    # Fit LDA model
    doc_topics = lda_model.fit_transform(dtm)
    
    # Get feature names
    feature_names = vectorizer.get_feature_names_out()
    
    # Get top words for each topic
    n_top_words = 10
    topics = []
    for topic_idx, topic in enumerate(lda_model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]
        topics.append({
            'topic_id': topic_idx,
            'words': top_words
        })
    
    # Create dictionary and corpus
    dictionary = Dictionary(processed_texts)
    corpus = [dictionary.doc2bow(text) for text in processed_texts]

    visualize_interactive_lda(lda_model, corpus, dictionary)
    
    return {
        'doc_topics': doc_topics,
        'topics': topics,
        'model': lda_model,
        'vectorizer': vectorizer,
        'processed_texts': processed_texts
    }

In [22]:
# Test with your data
def analyze_mental_health_topics(text_list):
    """
    Analyze mental health conversations and print topics
    """
    print("Starting analysis...")
    results = perform_topic_modeling(text_list, n_topics=10)
    
    print("\nDiscovered Topics:")
    for topic in results['topics']:
        print(f"\nTopic {topic['topic_id'] + 1}:")
        print(", ".join(topic['words']))
        
    return results

In [21]:
results = analyze_mental_health_topics(unique_contexts_list)

Starting analysis...

Discovered Topics:

Topic 1:
men, voices, therapy, abuse, remember, dealing, shake, learn, enough, far

Topic 2:
ask, drinking, see, lonely, daughter, husband, doctor, talking, pictures, second

Topic 3:
like, feel, want, always, know, time, never, things, really, talk

Topic 4:
done, depressed, thinks, act, attracted, dependent, causing, type, angry, concerned

Topic 5:
sexual, job, says, worker, fall, counseling, social, help, past, co

Topic 6:
depression, anxiety, names, help, stress, counseling, deal, mood, doctor, violent

Topic 7:
feel, like, get, want, know, years, love, time, relationship, think

Topic 8:
counselor, child, fight, wife, mother, lazy, client, parent, chance, sad

Topic 9:
difficult, feel, listen, order, trust, cannot, age, listening, approach, seems

Topic 10:
disorder, therapist, brother, friends, understand, two, gets, really, family, never


In [ ]:


# Assuming you have your LDA model, corpus, and dictionary


In [23]:


# Visualize with pyLDAvis
visualize_interactive_lda(lda_model, corpus, dictionary)

NameError: name 'processed_texts' is not defined